In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [3]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

この売上データに基づいて、いくつかの重要な傾向やインサイトを分析してみます。

### 1. カテゴリー別の売上分析
- **食品、衣服、日用品**の3つのカテゴリーがあります。それぞれのカテゴリー別に総売上や平均単価、利益率などを算出し、どのカテゴリーが最も強力かを把握しましょう。
- 例: `食品`の売上の合計、数量、平均単価を算出し、次に`衣服`、`日用品`と比較。

### 2. 利益率の分析
- 利益は `(単価 - 原価) × 数量` で算出できます。これを用いて、各商品やカテゴリーの利益率を計算しましょう。
- 高い利益率を持つ商品やカテゴリーを特定し、それに基づいてプロモーションや在庫戦略を見直します。

### 3. 売上の時間的なトレンド
- 売上日ごとの売上動向を視覚化し、特定の繁忙期や閑散期を特定します。時間に対する平均売上や累積売上のトレンドを示すグラフを作成することが有効です。

### 4. 人気商品と売上の集中
- 各商品コード（SKU）ごとの売上を集計し、トップセラーを特定します。また、売上が特定の少数の商品に集中している場合、それによるリスクを考慮し、商品ラインナップの多様化について議論することが重要です。

### 5. プロモーション効果の分析
- 特定の日に行ったプロモーションやキャンペーンの成果を分析し、売上データに顕著な変化が見られた場合、そのプロモーションの効果を評価します。

### 6. 顧客セグメンテーション
- 利用可能であれば、顧客情報を用いて、どの顧客セグメントがどのカテゴリーや商品を好む傾向にあるのかを分析することで、ターゲティングやパーソナライズプランを作成します。

### 7. 在庫管理
- 売上データに基づいて、各商品の売れ行きに応じた在庫管理戦略を見直す必要があります。需要が高い期間に対して在庫が不足しないよう、また処分すべき在庫を特定し、必要に応じて割引戦略を採用します。

これらの分析を通じて、企業は在庫管理、プロモーション戦略、製品開発の計画におけるデータ駆動型の意思決定が可能になります。さらに詳細な洞察や具体的な数値を得るために、実際のデータをもとにしたさらなる解析が望まれます。


In [4]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0              この売上データに基づいて、いくつかの重要な傾向やインサイトを分析してみます。
1                                                    
2                                  ### 1. カテゴリー別の売上分析
3   - **食品、衣服、日用品**の3つのカテゴリーがあります。それぞれのカテゴリー別に総売上や...
4        - 例: `食品`の売上の合計、数量、平均単価を算出し、次に`衣服`、`日用品`と比較。
5                                                    
6                                       ### 2. 利益率の分析
7   - 利益は `(単価 - 原価) × 数量` で算出できます。これを用いて、各商品やカテゴリ...
8   - 高い利益率を持つ商品やカテゴリーを特定し、それに基づいてプロモーションや在庫戦略を見直します。
9                                                    
10                                 ### 3. 売上の時間的なトレンド
11  - 売上日ごとの売上動向を視覚化し、特定の繁忙期や閑散期を特定します。時間に対する平均売上や...
12                                                   
13                                  ### 4. 人気商品と売上の集中
14  - 各商品コード（SKU）ごとの売上を集計し、トップセラーを特定します。また、売上が特定の少...
15                                                   
16                                ### 5. プロモーション効果の分析
17  - 特定の日に行ったプロモーションやキャンペーン

In [5]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [6]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")


処理を開始します。
Excelファイルに分析結果を保存しました。
